# PyTorch Classification Demo

## Notes

* Dataset: GroZi-120 dataset (http://grozi.calit2.net/), containing 120 product labels, 676 training data and 29 videos as evaluation data. The videos can be extracted as individual product images (this is provided in the dataset's website).
* Clone https://github.com/jonathan016/rpdr-config-results for dataset and result folder locations and put it at the same level as this project's location. **This is required for the notebook to run.**
    - Run `git clone https://github.com/jonathan016/rpdr-config-results.git` from your command line runner/terminal
* This notebook requires **at least** `Python 3.6.2` and the following library versions:
    - `notebook >= 5.2.2`
    - `torch >= 1.3.1`
    - `torchvision >= 0.4.2`
    - `pillow >= 6.2.1`
* To open this notebook in interactive mode, after cloning this repository (`git clone https://github.com/jonathan016/rpdr.git`), navigate to the cloned folder, then run `pip install notebook` and `jupyter notebook` from your command line runner/terminal.

## Library Imports

In [ ]:
# Python-provided libraries
import os
import sys
import time
from random import randint, uniform

In [ ]:
# External libraries
import torch
import torch.cuda as cuda
import torch.nn.functional as torch_fn

from PIL import ImageFilter
from torch.nn import CrossEntropyLoss, Linear, Module, Conv2d, MaxPool2d
from torch.optim import SGD
from torch.utils.data import DataLoader
from torchvision.models import vgg16
from torchvision.transforms import Compose, Resize, Grayscale, ToTensor, ColorJitter, Normalize, Lambda, \
    RandomResizedCrop, RandomErasing, RandomRotation, RandomPerspective, ToPILImage

## Pre-run Preparations

### Variable Definitions

In [ ]:
module_location = './'
train_root = '../rpdr-config-results/data/cropped'
eval_root = '../rpdr-config-results/data/in_situ_jpgs'
eval_indices = '../rpdr/val_test/recog_val_test.json'
eval_files = './val_test/recog_val_test_classes_files.json'
logfile = '../rpdr-config-results/results/ssd/base/3.log'
save = '../rpdr-config-results/results/ssd/base/3_300.pth.tar'
best = '../rpdr-config-results/results/ssd/base/3_300_best_model.pth.tar'
force_cuda = False  # TODO Set true if you have CUDA driver (only on NVIDIA GPUs)

### Import Custom Classes

In [ ]:
sys.path.append(module_location)
from utils.datasets import UsageBasedDataset, RecognitionDataset

## Declare Helper Functions

### Data Augmentation Functions

In [ ]:
def maybe_blur(image):
    return image.filter(ImageFilter.BoxBlur(randint(0, 7)))


def maybe_random_crop(image):
    if randint(0, 100) <= 35:
        return RandomResizedCrop(size=image.size, scale=(0.5, 1.0), ratio=(1., 1.))(image)
    return image


def maybe_random_erase(image):
    if randint(0, 100) <= 7:
        return ToPILImage()(RandomErasing(p=1.)(ToTensor()(image)))
    return image


def maybe_rotate(image):
    if randint(0, 100) <= 4:
        r = randint(0, 100)
        if r <= 20:
            return RandomRotation(degrees=90)(image)
        elif r <= 50:
            return RandomRotation(degrees=45)(image)
        else:
            return RandomRotation(degrees=30)(image)
    return image


def maybe_random_perspective(image):
    if randint(0, 100) <= 2:
        return RandomPerspective(distortion_scale=randint(4, 10) / 10, p=1.)(image)
    return image


def maybe_darken_a_lot(image):
    if randint(0, 100) <= 30:
        brightness = uniform(.5, .8)
        saturation = uniform(1., 1.5)
        return ColorJitter(brightness=(brightness, brightness), saturation=(saturation, saturation))(image)
    return image

### Logging Function

In [ ]:
def out(value):
    global logfile

    if logfile:
        print(value, file=open(logfile, 'a'))
    else:
        print(value)

### Evaluation Function

Useful for both validation and testing purposes

In [ ]:
def eval(model, criterion, loader):
    global force_cuda

    # Set evaluation flag on model. Makes all parameters non-trainable/non-adjustable during evaluation
    model.eval()

    # Initialize evaluation variables
    loss = 0.0
    acc = 0.0
    total = 0

    # Iterate through evaluation data
    for i, data in enumerate(loader):
        with torch.no_grad():
            inputs, labels = data

            # Transfer to CUDA device for faster execution, if CUDA device is available
            if force_cuda and cuda.is_available():
                inputs, labels = inputs.cuda(), labels.cuda()

            # Do forward pass, obtain predictions, then calculate loss
            outputs = model(inputs)
            _, prediction = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

            # Update training variables for logging purposes
            loss += loss.data.item()
            acc += torch.sum(prediction == labels.data).item()
            total += labels.size(0)

            # Clear memory of all processed data in this iteration
            del inputs, labels, outputs, prediction
            cuda.empty_cache()

    # Calculate average loss and average accuracy for logging purposes
    avg_loss = loss / total
    avg_acc = acc / total

    return avg_loss, avg_acc

### Training Function

In [ ]:
def train(model, optimizer, criterion, epoch, train_data, val_data, save_file, best_file):
    global force_cuda

    # Check for existing best model's weight file and load it if it exists
    if os.path.exists(best_file):
        out('Loading best model')
        loader = torch.load(best_file)
        model.load_state_dict(loader['state_dict'])
        best_val_acc = loader['best_val']
    else:
        best_val_acc = None

    # Set training flag on model. Makes all parameters adjustable during training
    model.train()

    # Initialize training variables
    total_loss = 0.0
    total_acc = 0.0
    total_img = 0
    iteration_losses = []

    # Iterate through training data
    for i, data in enumerate(train_data):
        inputs, labels = data

        # Transfer to CUDA device for faster execution, if CUDA device is available
        if force_cuda and cuda.is_available():
            inputs, labels = inputs.cuda(), labels.cuda()

        # Empty gradient/loss in the model parameters (encapsulated in the optimizer; search `Construct Optimizer` in this notebook).
        # Required by default in PyTorch to 'clean' the model's loss buffer before any new loss calculation and backpropagation
        optimizer.zero_grad()

        # Do forward pass, obtain predictions, then calculate loss
        outputs = model(inputs)
        _, predictions = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)

        # Compute loss for all model parameters
        loss.backward()
        # Do backpropagation (updates model parameters)
        optimizer.step()

        # Update training variables for logging purposes
        iteration_losses.append(loss.data.item())
        total_loss += loss.data.item()
        total_acc += torch.sum(predictions == labels.data).item()
        total_img += labels.size(0)

        # Clear memory of all processed data in this iteration
        del inputs, labels, outputs, predictions
        cuda.empty_cache()

    # Log training statistics for specified epoch
    out(f'Training #{epoch}: {total_acc / total_img} accuracy and {total_loss / total_img} loss')
    
    # Do validation
    val_loss, val_acc = eval(model, criterion, val_data)
    out(f'Validation #{epoch}: {val_acc} accuracy and {val_loss} loss')

    # Select and save best model's weight by validation accuracy
    if best_val_acc is None or best_val_acc < val_acc:
        best_val_acc = val_acc
        torch.save({
            'state_dict': model.state_dict(),
            'best_val': best_val_acc
        }, best_file)

    # Save per epoch
    saved_iteration_losses = iteration_losses
    if os.path.exists(save_file):
        saved_iteration_losses = torch.load(save_file)['iteration_losses']
        saved_iteration_losses.extend(iteration_losses)

    # Save checkpoint per training epoch
    torch.save({
        'model': model.state_dict(),
        'optimizer': optimizer.state_dict(),
        'criterion': criterion.state_dict(),
        'iteration_losses': saved_iteration_losses,
        'last_epoch': epoch
    }, save_file)

    return model, optimizer, criterion

## Construct Model

In [ ]:
vgg16_base = vgg16(pretrained=True)

# Replace final classifier (fully connected) layer to output desired number of class scores (in this case, 120)
vgg16_base.classifier[6] = Linear(4096, 120)

# You're familiar with this one.
if force_cuda and cuda.is_available():
    vgg16_base = vgg16_base.cuda()

## Training Preparation

### Construct Optimizer

In [ ]:
sgd = SGD(vgg16_base.parameters(), lr=.001, momentum=.9, weight_decay=.0005)

### Construct Loss Function

In [ ]:
crit = CrossEntropyLoss()

### Construct Training and Validation Data Feeder

Bear with me on this one.

We use the term `Data Feeder` as PyTorch uses the term `Dataset` as a class which cannot be used for mini-batch training. To use mini-batch training, we need to construct a `DataLoader` instance to be iterated on later in the `train` method.

#### Training Data

In [ ]:
# Create transformation to be used to augment the data on request (on-the-fly/on-line augmentation)
train_transform = Compose([
    Lambda(maybe_blur),
    Lambda(maybe_darken_a_lot),
    Lambda(maybe_rotate),
    Lambda(maybe_random_perspective),
    Lambda(maybe_random_crop),
    Lambda(maybe_random_erase),
    ColorJitter(brightness=(.1, .8), contrast=.05, saturation=.05, hue=.005),
    Resize(size=(300, 300)),
    Grayscale(num_output_channels=3),
    ToTensor(),
    Normalize((.5, .5, .5), (.5, .5, .5))
])

# Create the dataset. `UsageBasedDataset` is a custom class made for automatic balancing of data
train_dataset = UsageBasedDataset(train_root, usage=150, transform=train_transform)

# Create the `DataLoader` instance with batch size of 8. Google the rest of the method parameters for more information
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, pin_memory=True, drop_last=True, num_workers=4)

#### Validation Data

In [ ]:
# Create transformation to be used to transform the evaluation data on request (on-the-fly/on-line augmentation). This is not an
# augmentation transformation.
eval_transform = Compose([Resize(size=(300, 300)), Grayscale(num_output_channels=3), ToTensor(), Normalize((.5, .5, .5), (.5, .5, .5))])

# Create the dataset. `RecognitionDataset` is a custom class made for automatic selection of validation or testing data on
# GroZi-120 dataset
val_dataset = RecognitionDataset(eval_root, eval_indices, eval_files, RecognitionDataset.VAL, transform=eval_transform)

# Create the `DataLoader` instance with batch size of 1
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

## (Finally) Run Training

In [ ]:
# Load checkpoint, if exists before training. This alters the starting epoch as training have elapsed until the checkpoint's
# specified epoch
if os.path.exists(save):
    saved_checkpoint = torch.load(save)
    start_epoch = saved_checkpoint['last_epoch'] + 1
    sgd.load_state_dict(saved_checkpoint['optimizer'])
    crit.load_state_dict(saved_checkpoint['criterion'])
else:
    start_epoch = 1

In [ ]:
# Start training for 75 epochs. The number `76` is due to starting epoch starts at 1, so 1 + 75 = ? (You're smart)
start_time = time.time()
for epoch in range(start_epoch, 76):
    vgg16_base, sgd, crit = train(vgg16_base, sgd, crit, epoch, train_loader, val_loader, save, best)
end_time = time.time()
out(f'VGG base recognition training elapsed for {end_time - start_time} seconds')

## Test Model

### Construct Testing Data Feeder (Sounds familiar?)

In [ ]:
test_dataset = RecognitionDataset(eval_root, eval_indices, eval_files, RecognitionDataset.TEST, transform=eval_transform)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

### Run Testing

In [ ]:
# Load best model's weights to be used for testing
vgg16_base.load_state_dict(torch.load(best)['state_dict'])

In [ ]:
# Test
test_loss, test_acc = eval(vgg16_base, crit, test_loader)

In [ ]:
# Log results
out('\n=========')
out(f'Test Average Loss: {test_loss}')
out(f'Test Average Accuracy: {test_acc}')

# Final Notes

Goodluck!